https://habrahabr.ru/company/ods/blog/327250/

# Современные фреймворки градиентного бустинга

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def get_preprocessed_titanic():
    df = sns.load_dataset('titanic')

    X, y = df.drop(['survived'], axis=1), df.survived

    X["embarked"] = X["embarked"].fillna("S")
    X["fare"].fillna(X["fare"].median(), inplace=True)
    X["embark_town"] = X["embark_town"].fillna(X.embark_town.value_counts().index[0]) 
    
    average_age_titanic   = X["age"].mean()
    std_age_titanic       = X["age"].std()
    count_nan_age_titanic = X["age"].isnull().sum()
    rand_1 = np.random.randint(average_age_titanic - std_age_titanic, 
                               average_age_titanic + std_age_titanic, 
                               size=count_nan_age_titanic)
    X["age"][np.isnan(X["age"])] = rand_1

    X.drop(['deck', 'alive'], axis=1, inplace=True)
    
    return X, y

In [3]:
X, y = get_preprocessed_titanic()

In [4]:
X = pd.get_dummies(X)

In [5]:
from sklearn.model_selection import GridSearchCV, cross_val_score

In [6]:
SEED = 192837465
np.random.seed = SEED

## Sklearn

### AdaBoost

In [7]:
from sklearn.ensemble import AdaBoostClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [10]:
scores = cross_val_score(LogisticRegression(), X.values, y)
np.mean(scores), 2*np.std(scores)

(0.8136924803591471, 0.019309372092127106)

In [9]:
scores = cross_val_score(AdaBoostClassifier(LogisticRegression(), random_state=SEED), X, y)
np.mean(scores), 2*np.std(scores)

(0.80471380471380483, 0.025196346038881826)

In [9]:
scores = cross_val_score(AdaBoostClassifier(DecisionTreeClassifier(random_state=SEED), random_state=SEED), X, y)
np.mean(scores), 2*np.std(scores)

(0.78226711560044893, 0.039008186749680734)

In [10]:
scores = cross_val_score(AdaBoostClassifier(RandomForestClassifier(random_state=SEED), random_state=SEED), X, y)
np.mean(scores), 2*np.std(scores)

(0.77665544332211001, 0.020816203132425791)

##### Какие параметры для настройки есть?

* __`n_estimators`__ - сколько по сути итераций бустинга выполнять, обучая новый классификатор на остатках старого.
* __`learning_rate`__ - скорость обучения

##### Как настраивать?

* Используйте простой классификатор для бустинга.
* Этот простой классификатор не должен быть очень хорош в обобщающей способности, не нужно его сильно регуляризовать.

### GBM

In [13]:
from sklearn.ensemble import GradientBoostingClassifier

In [14]:
clf = GradientBoostingClassifier()

In [15]:
scores = cross_val_score(GradientBoostingClassifier(random_state=SEED), X, y)
np.mean(scores), 2*np.std(scores)

(0.82267115600448937, 0.042707738699080698)

##### Какие параметры для настройки?

Относящиеся к деревьям:

* __`min_samples_split`__ - б&#243;льшие значения, конечно, защищают от лишних сплитов по шумовым объектам, но слишком большие значения могут привести к недообучению. Можно опираться на значения ~0.5-1% от числа объектов в выборке. 
    * _Использовать CV для настройки._


* __`min_samples_leaf`__ - как и `min_samples_split`, является регуляризатором переобучения. В целом _меньшие значения можно использовать, когда речь идет о несбалансированных классах_, так как и области, содержащие эти редкие классы, небольшие.
    * _Использовать CV для настройки._
    

* __`min_weight_fraction_leaf`__ - похож на `min_samples_leaf`, но формулируется как _доля_ от общего числа объектов вместо _числа_. Настраивайте или этот параметр, или `min_samples_leaf` по усмотрению.
    * _Использовать CV для настройки._


* __`max_depth`__ - максимальная глубина дерева. Используется для контроля переобучения, так как б&#243;льшая глубина позволяет модели выучить неочевидные взаимосвязи относительно каждого из объектов выборки.
    * Обычно в диапазоне 3-10.
    * _Использовать CV для настройки._


* __`max_leaf_nodes`__ - максимальное число терминальных узлов, то есть листов дерева. Может быть использовано вместо `max_depth`. Т.к. деревья бинарные, дерево глубиной $N$ порождает _не более_ $2^N$ листов. Если задан этот параметр, GBM будет игнорировать `max_depth`.
    * _Использовать CV для настройки._


* __`max_features`__ - количество признаков, которое рассматривается при поиске лучшего сплита. Признаки при этом выбираются случайно. Как правило, $\sqrt{M}$, где $M$ - число признаков, работает отлично, но можно попробовать поднять эту отметку до 30-40% от общего числа признаков. 
    * `NB`: Большие значения могут привести к переобучению.
    
Относящиеся к бустингу:

* __`learning_rate`__ - данный параметр определяет вклад каждого из деревьев на финальное предсказание. 
    * Величина отражает мощность каждого вклада каждым деревом в финальный результат.
    * Меньшие значения более предпочтительны, так как они делают модель более робастной к специфическим параметрам дерева и таким образом повышают его обобщающую способность.
    * В то же время меньшие значения тянут за собой большее количество деревьев, что делает общую стоимость вычислений больше.


* __`n_estimators`__ - количество деревьев, которое нужно построить. 
    * Несмотря на то, что GBM достаточно робастен при большом количестве деревьев, он все равно может переобучиться. 
    * Зафиксируете определенный `learning_rate` и найдите оптимальное количество деревьев с помощью CV.


* __`subsample`__ - доля объектов, которая выделяется для обучения каждого из деревьев (подвыборка формируется простым случайным выбором).
    * Значения чуть меньшие 1 делают модель робастнее, так как уменьшают вариантивность подвыборок.
    * Обычно значение ~0.8 работает норм.

##### Как настраивать?

1. Выберите относительно большую скорость обучения (`learning rate`). Обычно это 0.1, но и иногда есть смысл выбрать из диапазона 0.05-0.2.

2. Выберите оптимальное количество деревьев при зафиксированной скорости обучения. Ориентируйтесь на 40-70. В том числе на этом шаге есть смысл взять такое количество, которое вас устраивает скоростью работы на конкретном компьютере, так как эту конфигурацию вам еще использовать для проверки множества сценариев (п.3).

3. Подберите оптимальные параметры деревьев, зафиксировав `learning_rate` и `n_estimators`.

4. Уменьшите `learning_rate`, увеличьте `n_estimators`, чтобы получить более робастные модели.

## XGBoost

In [16]:
from xgboost import XGBClassifier, XGBRegressor

In [19]:
clf = XGBClassifier()

In [18]:
scores = cross_val_score(XGBClassifier(seed=SEED), X, y)
np.mean(scores), 2*np.std(scores)

(0.81593714927048255, 0.024793178489758143)

Чем XGBoost интереснее, чем GBM?

1. Регуляризация из коробки.
2. Параллелится на ядра и процессоры на "ура".
3. Может работать с датафреймами с пропусками в значениях.
4. Предоставляет механизм кросс-валидации после каждой из итераций, что позволяет за единственный прогон понять, сколько итераций бустинга достаточно.

##### Какие параметры для настройки?

* __`booster [default=gbtree]`__ - тип модели. Всего 2 варианта: 
    * `gbtree` - деревья
    * `gblinear` - линейные модели

Дальше речь пойдет только о бустинге деревьев, так как они почти всегда ($\lim\rightarrow1)$ превосходят линейные модели.

Относящиеся к деревьям:

* __`eta [default=0.3]`__ - аналог `learning_rate`. Если вы используете sklearn-подобные обертки вроде XGBClassifier, то этот параметр и будет называться `learning_rate`.
    * Разумно перебирать из диапазона 0.01-0.2.


* __`min_child_weight [default=1]`__ - определяет минимальную сумму весов всех объектов в дочернем узле. Параметр похож на `min_child_leaf` в GBM, но не совсем: имеется в виду сумма весов объектов, в то время как в GBM имеется в виду просто количество объектов.
    * Позволяет управлять переобучением: б&#243;льшие значения не дают модели захватывать слишком частные взаимосвязи между объектами.
    * Слишком низкие значения ведут к недообучению.
    * _Использовать CV для настройки._
    

* __`max_depth [default=6]`__ - глубина, такая же, как и в GBM.
    * _Использовать CV для настройки._
    * Типичный диапазон: 3-10.
    

* __`max_leaf_nodes`__ - то же, что и в GBM. См. GBM.


* __`gamma [default=0]`__ - узел дерева будет разделен, если только в результате этого разделения будет уменьшено значение функции потерь. Параметр $\gamma$ задает границу величины этого уменьшения. Такой параметр делает алгоритм более консервативным. Значения, очевидно, опираются на выбранную в рамках задачи функцию потерь, и должен быть выбрать особо.


* __`max_delta_step [default=0]`__ - заставляет вес (считай, вклад) каждого дерева быть не меньше некоторого порога. В результате принимаемые обновления в бустинге становятся увереннее, и обновления более значимы. Тем-не-менее, этот параметр обычно оставляют равным значению по умолчанию, хотя в случаях _очень несбалансированной выборки_ может помочь.


* __`subsample [default=1]`__ - то же, что и в GBM. 
    * Обычно 0.5-1.


* __`colsample_bytree [default=1]`__ - то же, что и `max_features` в GBM.
    * В XGB пробуют в интервале 0.5-1.


* __`colsample_bylevel [default=1]`__ - обычно не используется, всю работу берет на себя предыдущий параметр.


* __`lambda [default=1]`__ - то же, что и $\alpha$ в $L_2$-регуляризации знакомого Ridge. Обычно не используется. Если вы используете sklearn-подобные обертки вроде XGBClassifier, то этот параметр и будет называться `reg_lambda`.


* __`alpha [default=0]`__ - то же, что и $\alpha$ в $L_1$-регуляризации знакомого Lasso. Может быть использован в очень высокоразмерном признаковом пространстве. Если вы используете sklearn-подобные обертки вроде XGBClassifier, то этот параметр и будет называться `reg_alpha`.


* __`scale_pos_weight [default=1]`__ - значения, большие чем 0, могут быть использованы в случае очень несбалансированных классов для более быстрого обучения.

##### Как настраивать?

1. Выберите относительно большую скорость обучения (`learning rate`). Обычно это 0.1, но и иногда есть смысл выбрать из диапазона 0.05-0.3.

2. Выберите оптимальное количество деревьев при зафиксированной скорости обучения. У XGB есть полезная функция `cv`, которая выполяет перекрестную проверку после каждой из итерации бустинга и возвращает оптимальное количество деревьев.

3. Подберите оптимальные параметры деревьев, зафиксировав `learning_rate` и `n_estimators`:
    * max_depth
    * min_child_weight 
    * gamma 
    * subsample 
    * colsample_bytree

4. Подберите параметры регуляризации:
    * reg_lambda
    * reg_alpha

5. Уменьшите `learning_rate`, выберите лучшие модели.

# LightGBM

In [21]:
from lightgbm import LGBMClassifier

In [22]:
scores = cross_val_score(LGBMClassifier(seed=SEED), X, y)
np.mean(scores), 2*np.std(scores)

(0.8271604938271605, 0.016797564025921096)

Чем интересен LightGBM?

* Меньшее потребление памяти
* Говорят, что по точности не хуже, а то и лучше
* Может вычисляться на GPU
* И считать очень большие объемы данных

LightGBM делает акцент на построении дерева по листьям, а не уровень за уровнем, за счет чего выигрывает в параллелизме. Поэтому авторы советуют в настройке сделать акцент на __`num_leaves`__.

Параметры в большинстве своем настраиваются по аналогии с XGB и GBM. Однако можно дать несколько рекомендаций:

##### Чтобы достичь большей точности

1. Используйте большее значение __`max_bin`__.
2. Уменьшайте __`learning_rate`__, увеличивайте __`num_iterations`__.
3. Можно попробовать увеличить __`num_leaves`__, но нужно смотреть за переобучением.

##### Чтобы бороться с переобучением

* Уменьшайте __`max_bin`__
* Уменьшайте __`num_leaves`__
* Используйте __`min_data_in_leaf`__ и __`min_sum_hessian_in_leaf`__
* Используйте бэггинг, задав параметры __`bagging_fraction`__ и __`bagging_freq`__
* Используйте подпространства признаков меньшей размерности через __`feature_fraction`__
* Регуляризация здесь осуществляется с помощью параметров __`lambda_l1`__, __`lambda_l2`__ и __`min_gain_to_split`__
* Задействуйте __`max_depth`__

# H2O

Легенда гласит, что оно еще быстрее LightGBM в распределенных вычислениях.

http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/gbm.html#defining-a-gbm-model

# CatBoost

https://habrahabr.ru/company/yandex/blog/333522/

In [24]:
from catboost import Pool, CatBoostClassifier

In [27]:
scores = cross_val_score(CatBoostClassifier(random_seed=SEED), X.values, y)
np.mean(scores), 2*np.std(scores)

(0.82042648709315369, 0.025983921218384328)

In [28]:
clf = CatBoostClassifier()

In [30]:
# clf.fit(X.values, y, plot=True)